#  Workshop Initiation

Welcome to our Text-to-SQL workshop! 
Have you ever struggled to bridge the gap between natural language queries and complex database interactions? As businesses seek more intuitive ways to extract insights from their data, Text-to-SQL technology has emerged as a critical solution for democratizing data access. In this workshop, we'll dive deep into practical prompt engineering techniques that transform natural language into precise SQL queries.

Through hands-on activities and expert guidance, you'll explore schema incorporation, prompt optimization, advanced templates, few-shot learning strategies, and innovative approaches like using Retrieval Augmented Generation (RAG) models. Our goal is to equip you with the skills to craft effective prompts that enable accurate and reliable natural language to SQL translation.

In [ ]:
region = 'us-west-2'
anthropic_sonnet = "anthropic.claude-3-sonnet-20240229-v1:0"
anthropic_haiku = "anthropic.claude-3-haiku-20240307-v1:0"
mistral7b = "mistral.mistral-7b-instruct-v0:2"
mistral_large = "mistral.mistral-large-2402-v1:0"

model_arn = f'arn:aws:bedrock:{region}::foundation-model/{anthropic_sonnet}'
session_id = None

%pip install -U "boto3" "botocore" "langchain-community"

import json
import boto3
from helpers import *


bedrock_runtime = boto3.client('bedrock-runtime')
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')

In [ ]:
def invoke_mistral_model(prompt, modelId):
    request_body = json.dumps({
        "prompt": "<s>[INST] " + prompt + " [/INST]",
        "max_tokens" : 400, 
        "temperature" : 0, 
        "top_p" : 0.9, 
        "top_k" : 50
    })
    response = bedrock_runtime.invoke_model(
            body = request_body,
            modelId = modelId
    )
    response_body = json.loads(response.get("body").read())
    return response_body.get('outputs')[0].get('text')

In [ ]:
 def invoke_anthropic_model(modelId, prompt):
        inferenceConfig = { 
            "maxTokens": 400,
            "temperature": 0
        }
        messages = [{
            "role": "user",
            "content": [{
                "text": prompt
            }]
        }]
        response = bedrock_runtime.converse(
            messages = messages,
            inferenceConfig = inferenceConfig,
            modelId = modelId
        )
        return response['output']['message']['content'][0]['text']

In [ ]:
file_name = "Schema-Rag"
def retrieveAndGenerate(input_text):
    input = {'text': input_text}
    config = {
        'type': 'EXTERNAL_SOURCES',
        'externalSourcesConfiguration': {
            'modelArn': model_arn,
            'sources': [
                {
                    "byteContent": {
                        "contentType": "text/plain",
                        "data": open(file_name, "rb").read(),
                        "identifier": file_name
                      },
                    "sourceType": "BYTE_CONTENT"  
                }
            ]
        }
    }
    response = bedrock_agent_runtime.retrieve_and_generate(
        input = input,
        retrieveAndGenerateConfiguration = config,
        )
    
    return response['output']['text']

# Database Creation

We are creating two databases for our workshop:

1. Anycompany: A standard database with comprehensive column descriptions
2. Chaos_anycompany: A database with modified column names and limited descriptive metadata
   
In the chaos_anycompany database, we discovered critical column name modifications that impact data interpretability. This exercise will examine the implications of reduced column descriptions and potential risks to data comprehension.

In [ ]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

def prepSQLDB():
    df_orders = pd.read_csv("orders.csv", delimiter=',')
    df_offices = pd.read_csv("offices.csv", delimiter=',')
    df_payments = pd.read_csv("payments.csv", delimiter=',')   
    df_products = pd.read_csv("products.csv", delimiter=',')
    df_productlines = pd.read_csv("productlines.csv", delimiter=',')  
    df_customers = pd.read_csv("customers.csv", delimiter=',')
    df_employees = pd.read_csv("employees.csv", delimiter=',')
    df_orderdetails = pd.read_csv("orderdetails.csv", delimiter=',')

    engine = create_engine("sqlite:///anycompany.db")

    df_orders.to_sql("orders", engine, index=False)
    df_offices.to_sql("offices", engine, index=False)
    df_payments.to_sql("payments", engine, index=False)
    df_products.to_sql("products", engine, index=False)
    df_productlines.to_sql("productlines", engine, index=False)
    df_customers.to_sql("customers", engine, index=False)
    df_employees.to_sql("employees", engine, index=False)
    df_orderdetails.to_sql("orderdetails", engine, index=False)
    

In [ ]:
def querySQL(statement):
    engine = create_engine("sqlite:///anycompany.db")
    db = SQLDatabase(engine=engine)
    return db.run(statement)


In [ ]:
prepSQLDB()

In [ ]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

def prepChaosSQLDB():
    df_orders = pd.read_csv("chaos_orders.csv", delimiter=',')
    df_offices = pd.read_csv("chaos_offices.csv", delimiter=',')
    df_payments = pd.read_csv("chaos_payments.csv", delimiter=',')   
    df_products = pd.read_csv("chaos_products.csv", delimiter=',')
    df_productlines = pd.read_csv("chaos_productlines.csv", delimiter=',')  
    df_customers = pd.read_csv("chaos_customers.csv", delimiter=',')
    df_employees = pd.read_csv("chaos_employees.csv", delimiter=',')
    df_orderdetails = pd.read_csv("chaos_orderdetails.csv", delimiter=',')

    engine = create_engine("sqlite:///chaos_anycompany.db")

    df_orders.to_sql("chaos_orders", engine, index=False)
    df_offices.to_sql("chaos_offices", engine, index=False)
    df_payments.to_sql("chaos_payments", engine, index=False)
    df_products.to_sql("chaos_products", engine, index=False)
    df_productlines.to_sql("chaos_productlines", engine, index=False)
    df_customers.to_sql("chaos_customers", engine, index=False)
    df_employees.to_sql("chaos_employees", engine, index=False)
    df_orderdetails.to_sql("chaos_orderdetails", engine, index=False)
    

In [ ]:
def querySQL_chaos(statement):
    engine = create_engine("sqlite:///chaos_anycompany.db")
    db = SQLDatabase(engine=engine)
    return db.run(statement)


In [ ]:
prepChaosSQLDB()

# Text2SQL 
  


# Scenario  1 - Basic Schema Incorporation
This scenario demonstrates how providing schema descriptions can significantly improve SQL query generation accuracy. This exercise highlights the critical role of contextual information in generating accurate SQL queries. By comparing the results, we can observe how schema incorporation enhances query precision and reduces errors caused by incorrect column name inference.

  - Model: Mistral AI - Mistral 7B Instruct
  - Techniques:
    - Schema Incorporation
    - Prompt Optimization
  - Use-Case: Listing San Francisco employees
    - Query 1: Without schema description
    - Query 2: With schema description
    - Query 3: With optimized schema description


In [ ]:
question = "List the full names of all San Francisco employees"

prompt = """
Your task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given tables called: productlines, products, offices, employees, customers, orders, payments, orderdetails


Write a SQL statement to answer the question:
""" + question + """

```sql
"""

In [ ]:
query_1_scenario_1 = invoke_mistral_model(prompt, mistral7b)
print(query_1_scenario_1)

In [ ]:
question = "List the full names of all San Francisco employees"

prompt = """
Your task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
productlines (
  productLine varchar(50),
  textDescription varchar(4000) DEFAULT NULL,
  htmlDescription mediumtext,
  image mediumblob,
  PRIMARY KEY (productLine)
);

products (
  productCode varchar(15),
  productName varchar(70) NOT NULL,
  productLine varchar(50) NOT NULL,
  productScale varchar(10) NOT NULL,
  productVendor varchar(50) NOT NULL,
  productDescription text NOT NULL,
  quantityInStock smallint(6) NOT NULL,
  buyPrice decimal(10,2) NOT NULL,
  MSRP decimal(10,2) NOT NULL,
  PRIMARY KEY (productCode),
  FOREIGN KEY (productLine) REFERENCES productlines (productLine)
);

offices (
  officeCode varchar(10),
  city varchar(50) NOT NULL,
  phone varchar(50) NOT NULL,
  addressLine1 varchar(50) NOT NULL,
  addressLine2 varchar(50) DEFAULT NULL,
  state varchar(50) DEFAULT NULL,
  country varchar(50) NOT NULL,
  postalCode varchar(15) NOT NULL,
  territory varchar(10) NOT NULL,
  PRIMARY KEY (officeCode)
);

employees (
  employeeNumber int,
  lastName varchar(50) NOT NULL,
  firstName varchar(50) NOT NULL,
  extension varchar(10) NOT NULL,
  email varchar(100) NOT NULL,
  officeCode varchar(10) NOT NULL,
  reportsTo int DEFAULT NULL,
  jobTitle varchar(50) NOT NULL,
  PRIMARY KEY (employeeNumber),
  FOREIGN KEY (reportsTo) REFERENCES employees (employeeNumber),
  FOREIGN KEY (officeCode) REFERENCES offices (officeCode)
);

customers (
  customerNumber int,
  customerName varchar(50) NOT NULL,
  contactLastName varchar(50) NOT NULL,
  contactFirstName varchar(50) NOT NULL,
  phone varchar(50) NOT NULL,
  addressLine1 varchar(50) NOT NULL,
  addressLine2 varchar(50) DEFAULT NULL,
  city varchar(50) NOT NULL,
  state varchar(50) DEFAULT NULL,
  postalCode varchar(15) DEFAULT NULL,
  country varchar(50) NOT NULL,
  salesRepEmployeeNumber int DEFAULT NULL,
  creditLimit decimal(10,2) DEFAULT NULL,
  PRIMARY KEY (customerNumber),
  FOREIGN KEY (salesRepEmployeeNumber) REFERENCES employees (employeeNumber)
);

payments (
  customerNumber int,
  checkNumber varchar(50) NOT NULL,
  paymentDate date NOT NULL,
  amount decimal(10,2) NOT NULL,
  PRIMARY KEY (customerNumber,checkNumber),
  FOREIGN KEY (customerNumber) REFERENCES customers (customerNumber)
);

orders (
  orderNumber int,
  orderDate date NOT NULL,
  requiredDate date NOT NULL,
  shippedDate date DEFAULT NULL,
  status varchar(15) NOT NULL,
  comments text,
  customerNumber int NOT NULL,
  PRIMARY KEY (orderNumber),
  FOREIGN KEY (customerNumber) REFERENCES customers (customerNumber)
);

orderdetails (
  orderNumber int,
  productCode varchar(15) NOT NULL,
  quantityOrdered int NOT NULL,
  priceEach decimal(10,2) NOT NULL,
  orderLineNumber smallint(6) NOT NULL,
  PRIMARY KEY (orderNumber,productCode),
  FOREIGN KEY (orderNumber) REFERENCES orders (orderNumber),
  FOREIGN KEY (productCode) REFERENCES products (productCode)
);

Write a SQL statement to answer the question:
""" + question + """

```sql
"""

In [ ]:
query_2_scenario_1 = invoke_mistral_model(prompt, mistral7b)
print(query_2_scenario_1)

In [ ]:
question = "List the full names of all San Francisco employees"

prompt = """
Your task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
offices (
  officeCode varchar(10),
  city varchar(50) NOT NULL,
  phone varchar(50) NOT NULL,
  addressLine1 varchar(50) NOT NULL,
  addressLine2 varchar(50) DEFAULT NULL,
  state varchar(50) DEFAULT NULL,
  country varchar(50) NOT NULL,
  postalCode varchar(15) NOT NULL,
  territory varchar(10) NOT NULL,
  PRIMARY KEY (officeCode)
);

employees (
  employeeNumber int,
  lastName varchar(50) NOT NULL,
  firstName varchar(50) NOT NULL,
  extension varchar(10) NOT NULL,
  email varchar(100) NOT NULL,
  officeCode varchar(10) NOT NULL,
  reportsTo int DEFAULT NULL,
  jobTitle varchar(50) NOT NULL,
  PRIMARY KEY (employeeNumber),
  FOREIGN KEY (reportsTo) REFERENCES employees (employeeNumber),
  FOREIGN KEY (officeCode) REFERENCES offices (officeCode)
);

Write a SQL statement to answer the question:
""" + question + """

```sql
"""

In [ ]:
query_3_scenario_1 = invoke_mistral_model(prompt, mistral7b)
print(query_3_scenario_1)

# Queries Execution



In [ ]:
res = querySQL(query_1_scenario_1)
prettyJSON(res)

In [ ]:
res = querySQL(query_2_scenario_1)
prettyJSON(res)

In [ ]:
res = querySQL(query_3_scenario_1)
prettyJSON(res)

## Querty Generation Results
- Query 1: Incorrect - Used wrong column names
- Query 2: Correct - Generated after incorporating schema description
- Query 3: Correct - Generated with optimized prompt (reduced irrelevant tables and columns)

## Key Findings
- Schema incorporation significantly improves query accuracy
- Optimizing the prompt by reducing irrelevant tables and columns for our question

## Scenario  2 - Enhancing SQL Query Accuracy with Table Relationships

This scenario demonstrates how including table relationship information improves SQL query generation accuracy. While table and column names provide a foundation, understanding table relationships significantly enhances query quality.

  - Model: Anthropic Haiku
  - Techniques:
    - Schema Incorporation
  - Use-Case: List all San Francisco employees and their manager's name
    - Query 1: With schema description
    - Query 2: With Schema description and table's relations 
    
The second query, enriched with table relationship details, produces a more precise and efficient result. This approach improves query quality and reduces potential errors in complex database operations.
By comparing these queries, you'll see how comprehensive database structure knowledge leads to better SQL generation. 


In [ ]:
question = "List the full names of all San Francisco office's employees, along with their managers full names"

prompt = """
Your task is to generate the simplest and most effective SQL query.
Given these tables:
offices (
  officeCode,
  city,
  phone,
  addressLine1,
  addressLine2,
  state,
  country,
  postalCode,
  territory
);

employees (
  employeeNumber,
  lastName,
  firstName,
  extension,
  email,
  officeCode,
  reportsTo,
  jobTitle
);

Please be aware of tables columns, sqllite syntax, and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_1_scenario_2 = invoke_anthropic_model(anthropic_haiku, prompt)
print(query_1_scenario_2)

In [ ]:
question = "List the full names of all San Francisco office's employees, along with their managers full names"

prompt = """
Your task is to generate the simplest and most effective SQL query.
Given these tables:
offices (
  officeCode,
  city,
  phone,
  addressLine1,
  addressLine2,
  state,
  country,
  postalCode,
  territory,
  PRIMARY KEY (officeCode)
);

employees (
  employeeNumber,
  lastName,
  firstName,
  extension,
  email,
  officeCode,
  reportsTo,
  jobTitle,
  PRIMARY KEY (employeeNumber),
  FOREIGN KEY (reportsTo) REFERENCES employees (employeeNumber),
  FOREIGN KEY (officeCode) REFERENCES offices (officeCode)
);

Please be aware of tables columns, sqllite syntax, and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_2_scenario_2 = invoke_anthropic_model(anthropic_haiku, prompt)
print(query_2_scenario_2)

# Queries Execution



In [ ]:
res = querySQL(query_1_scenario_2)
prettyJSON(res)

In [ ]:
res = querySQL(query_2_scenario_2)
prettyJSON(res)

## Querty Generation Results
- Query 1: Incorrect - Returns all San Francisco employees with their managers' names, excluding the Chief Executive Officer (CEO) who has no manager.
- Query 2: Correct - Returns all San Francisco employees with their managers' names, including the CEO

## Key Findings
- Basic schema descriptions may omit important data (e.g., CEO in Result 1).
- Including table relationships produces more comprehensive results (Result 2).

Accurate queries require understanding of database structure beyond simple table and column names.
This approach ensures all relevant data is retrieved, even in cases with unique hierarchical structures. By specifying table relationships, we create more robust and reliable SQL queries for complex database operations.

## Scenario  3 - Improving Query Accuracy with Descriptive Column Names
This scenario demonstrates how descriptive column names and targeted schema descriptions enhance SQL query accuracy

Have you ever struggled to write accurate SQL queries because of confusing database column names? You're not alone. In this post, we'll explore how to improve query accuracy and efficiency, even when working with poorly named columns.

Many databases use cryptic column names that obscure data meaning. For example for the same Customers table we have different column name:
'anycompany' database: salesRepEmployeeNumber
'chaos_anycompany' database: enumber

  - Model: Mistral Large
  - Techniques:
    - Schema Incorporation
  - Use-Case: Count customers that purchased motorcycles
    - Query 1: Using anycompany database and schema description
    - Query 2: Using chaos_anycompany database and schema description
    - Query 3: Using chaos_anycompany database and schema description and relevant columns descripion 

As a bonus exercise:
Consider optimizing the prompt to get the same query, such as removing irrelevant columns

In [ ]:
question = "How many customers purchased motorcycles"

prompt = """
Your task is to generate the simplest and most effective SQL query.
Given these tables:

products (
  productCode,
  productName,
  productLine,
  productScale,
  productVendor,
  productDescription,
  quantityInStock,
  buyPrice,
  MSRP
);

customers (
  customerNumber,
  customerName,
  contactLastName,
  contactFirstName,
  phone,
  addressLine1,
  addressLine2,
  city,
  state,
  postalCode,
  country,
  salesRepEmployeeNumber,
  creditLimit
);

orders (
  orderNumber,
  orderDate,
  requiredDate,
  shippedDate,
  status,
  comments,
  customerNumber
);

orderdetails(
  orderNumber,
  productCode,
  quantityOrdered,
  priceEach,
  orderLineNumber 
);

Please be aware of tables columns, sqllite syntax!! and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_1_scenario_3 = invoke_mistral_model(prompt, mistral_large)
print(query_1_scenario_3)

In [ ]:
question = "How many customers purchased motorcycles"

prompt = """
Your task is to generate the simplest and most effective SQL query.
Given these tables:

chaos_products (
  productCode,
  productName,
  pline,
  productScale,
  productVendor,
  productDescription,
  quantityInStock,
  buyPrice,
  MSRP
);

chaos_customers (
  customerNumber,
  customerName,
  contactLastName,
  contactFirstName,
  phone,
  addressLine1,
  addressLine2,
  city,
  state,
  postalCode,
  country,
  enumber,
  creditLimit
);

chaos_orders (
  orderNumber,
  orderDate,
  requiredDate,
  shippedDate,
  status,
  comments,
  cnumber
);

chaos_orderdetails(
  orderNumber,
  pcode,
  quantityOrdered,
  priceEach,
  orderLineNumber 
);

Please be aware of tables columns, sqllite syntax!! and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_2_scenario_3 = invoke_mistral_model(prompt, mistral_large)
print(query_2_scenario_3)

In [ ]:
question = "How many customers purchased motorcycles"

prompt = """
Your task is to generate the simplest and most effective SQL query.
Given these tables:

chaos_products (
  productCode,
  productName,
  pline -- Product line such as Ships, Classic Cars, Motorcycles, etc
  productScale,
  productVendor,
  productDescription,
  quantityInStock,
  buyPrice,
  MSRP
);

chaos_customers (
  customerNumber,
  customerName,
  contactLastName,
  contactFirstName,
  phone,
  addressLine1,
  addressLine2,
  city,
  state,
  postalCode,
  country,
  enumber,
  creditLimit
);

chaos_orders (
  orderNumber,
  orderDate,
  requiredDate,
  shippedDate,
  status,
  comments,
  cnumber
);

chaos_orderdetails(
  orderNumber,
  pcode,
  quantityOrdered,
  priceEach,
  orderLineNumber 
);

Please be aware of tables columns, sqllite syntax!! and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_3_scenario_3 = invoke_mistral_model(prompt, mistral_large)
print(query_3_scenario_3)

# Queries Execution



In [ ]:
res = querySQL(query_1_scenario_3)
prettyJSON(res)

In [ ]:
res = querySQL_chaos(query_2_scenario_3)
prettyJSON(res)

In [ ]:
res = querySQL_chaos(query_3_scenario_3)
prettyJSON(res)

## Querty Generation Results
- Query 1: Correct - Returns the number of customers that purchased motorcycles - 55
- Query 2: Incorrect - Returns false answer - 0
- Query 3: Correct - Returns the number of customers that purchased motorcycles - 55

## Key Findings
- The names of columns in our organization's database are not always descriptive enough for the model to understand the meaning of the column. Therefore we need to describe the relevant columns in our prompt

## Scenario  4 - Model Size Impact on SQL Query Accuracy
This scenario demonstrates how a smaller model is not capable of providing an accurate SQL query for a given question, whereas a larger model is capable of providing the correct query.

  - Models: 
    - Mistral AI - Mistral large
    - Anthropic Claude
  - Use Case: List all customers who have ordered products from all product lines
    - Qurey 1: Using Mistral 7B
    - Qurey 2: Using Mistral large
    - Query 3: Using Anthropic Claude
  
Bonus Exercise: 
Enhance the prompt to assist Mistral Large in providing an accurate response


In [ ]:
## TIP: Few Shots

In [ ]:
question = "List the names of all customers who have purchased at least one product from each line of products"

prompt = """
Your task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
productlines -- This table consist of all product lines
(
  productLine varchar(50),
  textDescription varchar(4000) DEFAULT NULL,
  htmlDescription mediumtext,
  image mediumblob,
  PRIMARY KEY (productLine)
);

products (
  productCode varchar(15),
  productName varchar(70) NOT NULL,
  productLine varchar(50) NOT NULL -- product line such as Ships, Classic Cars, etc,
  productScale varchar(10) NOT NULL,
  productVendor varchar(50) NOT NULL,
  productDescription text NOT NULL,
  quantityInStock smallint(6) NOT NULL,
  buyPrice decimal(10,2) NOT NULL,
  MSRP decimal(10,2) NOT NULL,
  PRIMARY KEY (productCode),
  FOREIGN KEY (productLine) REFERENCES productlines (productLine)
);

customers (
  customerNumber int,
  customerName varchar(50) NOT NULL,
  contactLastName varchar(50) NOT NULL,
  contactFirstName varchar(50) NOT NULL,
  phone varchar(50) NOT NULL,
  addressLine1 varchar(50) NOT NULL,
  addressLine2 varchar(50) DEFAULT NULL,
  city varchar(50) NOT NULL,
  state varchar(50) DEFAULT NULL,
  postalCode varchar(15) DEFAULT NULL,
  country varchar(50) NOT NULL,
  salesRepEmployeeNumber int DEFAULT NULL,
  creditLimit decimal(10,2) DEFAULT NULL,
  PRIMARY KEY (customerNumber),
  FOREIGN KEY (salesRepEmployeeNumber) REFERENCES employees (employeeNumber)
);

orders (
  orderNumber int,
  orderDate date NOT NULL,
  requiredDate date NOT NULL,
  shippedDate date DEFAULT NULL,
  status varchar(15) NOT NULL,
  comments text,
  customerNumber int NOT NULL,
  PRIMARY KEY (orderNumber),
  FOREIGN KEY (customerNumber) REFERENCES customers (customerNumber)
);

orderdetails (
  orderNumber int,
  productCode varchar(15) NOT NULL,
  quantityOrdered int NOT NULL,
  priceEach decimal(10,2) NOT NULL,
  orderLineNumber smallint(6) NOT NULL,
  PRIMARY KEY (orderNumber,productCode),
  FOREIGN KEY (orderNumber) REFERENCES orders (orderNumber),
  FOREIGN KEY (productCode) REFERENCES products (productCode)
);

Please be aware of tables,columns, and primary and foreign key.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement and syntax to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_1_scenario_4 = invoke_mistral_model(prompt, mistral7b)
print(query_1_scenario_4)

In [ ]:
query_2_scenario_4 = invoke_mistral_model(prompt, mistral_large)
print(query_2_scenario_4)

In [ ]:
query_3_scenario_4 = invoke_anthropic_model(anthropic_haiku, prompt)
print(query_2_scenario_4)

# Queries Execution



In [ ]:
res = querySQL(query_1_scenario_4)
prettyJSON(res)

In [ ]:
res = querySQL(query_2_scenario_4)
prettyJSON(res)

In [ ]:
res = querySQL( query_3_scenario_4 )
prettyJSON(res)

## Results
- Result 1: with Mistral7b Instruct - wrong query, wrong results
- Result 2: With Mistral Large - correct query, not optimized, correct results
- Result 3: With Antropic Claude Haiku - correct query, correct results

## Key Findings
- Larger language models like Antropic Claude Haiku have better capabilities in understanding context, capturing relationships between entities, and performing complex reasoning, which are crucial for accurately translating natural language queries into optimized SQL statements based on a given database schema.

## Scenario  5 - Enhancing SQL Query Generation with Single Document RAG and Database Schema
This scenario demonstrates the use of a Single Document Retrieval Augmented Generation (RAG) model with database schema to improve SQL query generation.
The RAG model provides relevant schema details, which are then incorporated into the prompt for the main model. This approach simplifies the integration of schema information, especially for large and complex databases.

  - Model: 
    - Anthropic Claude 3 Sonnet
  - Use Cases: Scenarios #2 #3 #4
    - Qurey 1: List customer names who have ordered products from all product lines
    - Query 2: Count customers that purchased motorcycles
    - Query 3: List all San Francisco employees and their manager's name
  - Method: Single Document Retrieval Augmented Generation (RAG) with Bedrock Knowledge Bases
  - Reference: https://aws.amazon.com/blogs/machine-learning/knowledge-bases-in-amazon-bedrock-now-simplifies-asking-questions-on-a-single-document/



In [ ]:
question1 = "List the names of all customers who have purchased at least one product from each line of products"

In [ ]:
question2 = "How many customers purchased motorcycles"

In [ ]:
question3 = "List the full names of all San Francisco office's employees, along with their managers full names"

In [ ]:
question_for_rag_template = "Provide schema representation containing table name, column names, and relationships between tables for the following question: {0}  The output should be a clean text containing the table's description. Don't write any explanations and comments"

## Crafting the questions for RAG

In [ ]:
question1_for_rag = question_for_rag_template.format(question1)

In [ ]:
question2_for_rag = question_for_rag_template.format(question2)

In [ ]:
question3_for_rag = question_for_rag_template.format(question3)

## Retrieving schema description from the RAG output

In [ ]:
res1 = retrieveAndGenerate(question1_for_rag)
prettyJSON(res1)

In [ ]:
res2 = retrieveAndGenerate(question2_for_rag)
prettyJSON(res2)

In [ ]:
res3 = retrieveAndGenerate(question3_for_rag)
prettyJSON(res3)

## Consolidating the queastions and the RAG Outputs into cohesive prompts

In [ ]:
prompt1 = """
Your task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
    """ + res1 + """

Please be aware of tables,columns, and primary and foreign key.
Dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run against the sqlite3 database
Construct only the valid sqlite3 SQL statement and syntax to answer the following the question:
""" + question1 + """

sql
"""

prompt2 = """
Your task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
    """ + res2 + """

Please be aware of tables,columns, and primary and foreign key.
Dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run against the sqlite3 database
Construct only the valid sqlite3 SQL statement and syntax to answer the following the question:
""" + question2 + """

sql
"""

prompt3 = """
Your task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
    """ + res3 + """

Please be aware of tables columns, sqllite syntax, and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question3 + """

sql
"""

## SQL Query Generation

In [ ]:
query_1_scenario_5 = invoke_anthropic_model(anthropic_haiku, prompt1)
print(query_1_scenario_5)

In [ ]:
query_2_scenario_5 = invoke_anthropic_model(anthropic_haiku, prompt2)
print(query_2_scenario_1)

In [ ]:
print (prompt3)
query_3_scenario_5 = invoke_anthropic_model(anthropic_haiku, prompt3)
print(query_3_scenario_5)

# Queries Execution
- Please note the results are identical to the results we received the previous scenarios

In [ ]:
res = querySQL(query_1_scenario_5)
prettyJSON(res)

In [ ]:
res = querySQL(query_2_scenario_1)
prettyJSON(res)

In [ ]:
res = querySQL(query_3_scenario_5)
prettyJSON(res)

## Test your knowledge 
Apply the learned techniques to write multiple prompts for the following questions using any of the workshop models:

 - What are the names of all the product lines?
 - List all the products with their names, scales, and vendors.
 - Show the first and last names of all employe
 - Find the names and credit limits of customers who have a sales representative assigned.
 - Retrieve the order numbers, order dates, and statuses for orders placed in the year 2022.
 - Get the product codes and quantities ordered for a specific order number.
 - Calculate the total revenue (quantity ordered * price each) for each product line, ordered by revenue in descending order.
 - Retrieve the names of customers who have placed orders for products from all product lines, along with the number of distinct product lines they have ordered from.


In [ ]:
question = ""

prompt = """
Your task is to generate the simplest and most effective SQL query.
Given these tables:



Please be aware of tables columns, sqllite syntax, and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_rag = question_for_rag_template.format(question)

In [ ]:
res = retrieveAndGenerate(query_rag)
prettyJSON(res)

In [ ]:
query_1 = invoke_mistral_model(prompt, mistral7b)
print(query_1)

In [ ]:
query_2 = invoke_anthropic_model(anthropic_haiku, prompt)
print(query_2)

# Queries Execution



In [ ]:
res = querySQL(query_1)
prettyJSON(res)

In [ ]:
res = querySQL(query_2)
prettyJSON(res)

## Thank You